In [1]:
import sys
sys.executable


'/Users/surbhisharma/rag_assesment/.venv1/bin/python'

In [12]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [14]:
!pip install --upgrade transformers


You should consider upgrading via the '/Users/surbhisharma/rag_assesment/.venv1/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip3 install transformers datasets peft torch


You should consider upgrading via the '/Users/surbhisharma/rag_assesment/.venv1/bin/python3 -m pip install --upgrade pip' command.


In [4]:
!pip3 install ipykernel



  Using cached ipykernel-6.30.1-py3-none-any.whl (117 kB)
  Using cached jupyter_client-8.6.3-py3-none-any.whl (106 kB)
  Using cached comm-0.2.3-py3-none-any.whl (7.3 kB)
  Using cached appnope-0.1.4-py2.py3-none-any.whl (4.3 kB)
  Using cached tornado-6.5.2-cp39-abi3-macosx_10_9_x86_64.whl (440 kB)
  Using cached nest_asyncio-1.6.0-py3-none-any.whl (5.2 kB)
  Using cached traitlets-5.14.3-py3-none-any.whl (85 kB)
  Using cached pyzmq-27.0.2-cp39-cp39-macosx_10_15_universal2.whl (1.3 MB)
  Using cached debugpy-1.8.16-cp39-cp39-macosx_14_0_x86_64.whl (2.1 MB)
  Using cached ipython-8.18.1-py3-none-any.whl (808 kB)
  Using cached matplotlib_inline-0.1.7-py3-none-any.whl (9.9 kB)
  Using cached jupyter_core-5.8.1-py3-none-any.whl (28 kB)
  Using cached prompt_toolkit-3.0.52-py3-none-any.whl (391 kB)
  Using cached jedi-0.19.2-py2.py3-none-any.whl (1.6 MB)
  Using cached stack_data-0.6.3-py3-none-any.whl (24 kB)
  Using cached pexpect-4.9.0-py2.py3-none-any.whl (63 kB)
  Using cached deco

In [3]:
from datasets import Dataset


/Users/surbhisharma/rag_assesment/.venv1/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/surbhisharma/rag_assesment/.venv1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from datasets import Dataset

data = {
    "question": ["What is the capital of France?", "Who wrote '1984'?", "What is 2 + 2?"],
    "answer": ["Paris", "George Orwell", "4"]
}

dataset = Dataset.from_dict(data)


In [38]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType

# Load model and tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
tokenizer.pad_token = '[PAD]'
# Configure LoRA adapter
from peft import LoraConfig, get_peft_model

# Example: attach LoRA to a GPT2 model
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["c_attn"],  # GPT2 attention layers
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # check which params are trainable


trainable params: 73,728 || all params: 81,987,072 || trainable%: 0.0899


/Users/surbhisharma/rag_assesment/.venv1/lib/python3.9/site-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [39]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)  # LoRA layers should have True


base_model.model.transformer.wte.weight False
base_model.model.transformer.wpe.weight False
base_model.model.transformer.h.0.ln_1.weight False
base_model.model.transformer.h.0.ln_1.bias False
base_model.model.transformer.h.0.attn.c_attn.base_layer.weight False
base_model.model.transformer.h.0.attn.c_attn.base_layer.bias False
base_model.model.transformer.h.0.attn.c_attn.lora_A.default.weight True
base_model.model.transformer.h.0.attn.c_attn.lora_B.default.weight True
base_model.model.transformer.h.0.attn.c_proj.weight False
base_model.model.transformer.h.0.attn.c_proj.bias False
base_model.model.transformer.h.0.ln_2.weight False
base_model.model.transformer.h.0.ln_2.bias False
base_model.model.transformer.h.0.mlp.c_fc.weight False
base_model.model.transformer.h.0.mlp.c_fc.bias False
base_model.model.transformer.h.0.mlp.c_proj.weight False
base_model.model.transformer.h.0.mlp.c_proj.bias False
base_model.model.transformer.h.1.ln_1.weight False
base_model.model.transformer.h.1.ln_1.bias 

In [40]:
def tokenize_function(examples):
    # Example: Concatenate question and answer
    model_inputs = tokenizer(examples['question'], truncation=True, padding='max_length', max_length=128)
    
    # Use the same tokens for labels (for causal LM)
    model_inputs['labels'] = model_inputs['input_ids']
    return model_inputs


tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 3/3 [00:00<00:00, 224.33 examples/s]


In [15]:
pip show transformers


Name: transformers
Version: 4.56.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /Users/surbhisharma/rag_assesment/.venv1/lib/python3.9/site-packages
Requires: packaging, tqdm, numpy, pyyaml, huggingface-hub, tokenizers, safetensors, requests, regex, filelock
Required-by: sentence-transformers, peft
Note: you may need to restart the kernel to use updated packages.


In [41]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./lora_distilgpt2",
    learning_rate=2e-5,
    eval_strategy="epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps=1,                # Log every step
    logging_strategy="steps",       # Use step-based logging
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets
)

trainer.train()


Epoch,Training Loss,Validation Loss
1,14.005000,13.872079
2,14.167600,13.867644
3,13.374300,13.863847
4,13.243600,13.860340
5,13.961400,13.857364
6,13.865600,13.854894
7,13.968300,13.852844
8,13.645400,13.851298
9,13.531600,13.850308
10,13.564400,13.849885


TrainOutput(global_step=20, training_loss=13.642737102508544, metrics={'train_runtime': 7.3232, 'train_samples_per_second': 4.097, 'train_steps_per_second': 2.731, 'total_flos': 981561507840.0, 'train_loss': 13.642737102508544, 'epoch': 10.0})

In [42]:
model.save_pretrained("./lora_distilgpt2")
tokenizer.save_pretrained("./lora_distilgpt2")


/Users/surbhisharma/rag_assesment/.venv1/lib/python3.9/site-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


('./lora_distilgpt2/tokenizer_config.json',
 './lora_distilgpt2/special_tokens_map.json',
 './lora_distilgpt2/vocab.json',
 './lora_distilgpt2/merges.txt',
 './lora_distilgpt2/added_tokens.json',
 './lora_distilgpt2/tokenizer.json')

In [43]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Load base model and tokenizer
base_model = AutoModelForCausalLM.from_pretrained("distilgpt2")
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

# Set pad_token to eos_token
tokenizer.pad_token = tokenizer.eos_token

# Resize embeddings if needed
base_model.resize_token_embeddings(len(tokenizer))

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, "./lora_distilgpt2", ignore_mismatched_sizes=True)

# Prepare input with attention mask
inputs = tokenizer("What is the capital of France?", return_tensors="pt", padding=True)
input_ids = inputs.input_ids
attention_mask = inputs.attention_mask

prompt = "Answer the following question with only the factual answer:\nQ: What is the capital of France?\nA:"
inputs = tokenizer(prompt, return_tensors="pt")
output_ids = model.generate(inputs.input_ids, max_new_tokens=10, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))



/Users/surbhisharma/rag_assesment/.venv1/lib/python3.9/site-packages/peft/utils/save_and_load.py:546: UserWarning: Some weights of PeftModelForCausalLM were not initialized from the model checkpoint and are being ignored because you passed `ignore_mismatched_sizes=True`: - base_model.model.lm_head.weight: found shape torch.Size([50258, 768]) in the checkpoint and torch.Size([50257, 768]) in the model instantiated
- base_model.model.transformer.wte.weight: found shape torch.Size([50258, 768]) in the checkpoint and torch.Size([50257, 768]) in the model instantiated.
  warnings.warn(msg)


Answer the following question with only the factual answer:
Q: What is the capital of France?
A: The capital of France is the capital of France.
